In [122]:
import numpy as np
import pickle
import torch
import torch.nn as nn
from torch.autograd import Variable
from torchtext import data
from torchtext import vocab
import torch.backends.cudnn as cudnn
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import revtok
from torchtext.data import BucketIterator
from matplotlib import pylab
from sklearn.manifold import TSNE
import random

In [123]:
# Fields for articles(text) and titles(summary)
text = data.ReversibleField(sequential=True, tokenize=None, include_lengths=True)
summary = data.ReversibleField(sequential=True, tokenize=None, 
                               include_lengths=True, eos_token='<eos>')

# Creating training and validation datasets
train, valid = data.TabularDataset.splits(
    path='./processed_data/', train='train_dataset.csv',
    validation='valid_dataset.csv', skip_header=True, 
    format='csv', fields=[('article', text), ('title', summary)])

In [124]:
# Word embedidngs to build vocabulary
#input_vectors = vocab.Vectors('crawl-300d-2M-input.vec')
#output_vectors = vocab.Vectors('crawl-300d-2M-output.vec')

#text.build_vocab(train, vectors=input_vectors)
text.build_vocab(train, vectors='glove.6B.300d')

#summary.build_vocab(train, vectors=input_vectors)
summary.build_vocab(train, vectors='glove.6B.300d')

# Number of dimensions of embeddings
num_dims = 300

In [125]:
batch_size = 128

# Iterator for generating batches
train_iter, valid_iter = BucketIterator.splits(
    (train, valid), 
    (batch_size, 1),
    device=0,
    #sort=True, # sort in ascending order of lenghts
    sort_key=lambda x: len(x.article),
    sort_within_batch=True, # sort each batch in descending order of article length
    repeat=True)

In [121]:
count = 0
for batch in train_iter:
    count += 1
    print(count, batch.article[0].data.cpu().numpy().shape)
    #print(count, torch.max(batch.article[1]))

1 (32, 128)
2 (35, 128)
3 (38, 128)
4 (41, 128)
5 (45, 128)
6 (49, 128)
7 (53, 128)
8 (58, 128)
9 (62, 128)
10 (66, 128)
11 (69, 128)
12 (73, 128)
13 (76, 128)
14 (79, 128)
15 (80, 128)
16 (80, 128)
17 (80, 128)
18 (80, 128)
19 (80, 128)
20 (80, 128)
21 (80, 128)
22 (80, 128)
23 (80, 128)
24 (80, 128)
25 (80, 128)
26 (80, 128)
27 (80, 128)
28 (80, 128)
29 (80, 128)
30 (80, 128)
31 (80, 128)
32 (80, 128)
33 (80, 128)
34 (80, 128)
35 (80, 128)
36 (80, 128)
37 (80, 128)
38 (80, 128)
39 (80, 128)
40 (80, 128)
41 (80, 128)
42 (80, 128)
43 (80, 128)
44 (80, 128)
45 (80, 128)
46 (80, 128)
47 (80, 128)
48 (80, 128)
49 (80, 128)
50 (80, 128)
51 (80, 128)
52 (80, 128)
53 (80, 128)
54 (80, 128)
55 (80, 128)
56 (80, 128)
57 (80, 128)
58 (80, 128)
59 (80, 128)
60 (80, 128)
61 (80, 128)
62 (80, 128)
63 (80, 128)
64 (80, 128)
65 (80, 128)
66 (80, 128)
67 (80, 128)
68 (80, 128)
69 (80, 128)
70 (80, 128)
71 (80, 128)
72 (80, 128)
73 (80, 128)
74 (80, 128)
75 (80, 128)
76 (80, 128)
77 (80, 128)
78 (80, 

635 (80, 128)
636 (80, 128)
637 (80, 128)
638 (80, 128)
639 (80, 128)
640 (80, 128)
641 (80, 128)
642 (80, 128)
643 (80, 128)
644 (80, 128)
645 (80, 128)
646 (80, 128)
647 (80, 128)
648 (80, 128)
649 (80, 128)
650 (80, 128)
651 (80, 128)
652 (80, 128)
653 (80, 128)
654 (80, 128)
655 (80, 128)
656 (80, 128)
657 (80, 128)
658 (80, 128)
659 (80, 128)
660 (80, 128)
661 (80, 128)
662 (80, 128)
663 (80, 128)
664 (80, 128)
665 (80, 128)
666 (80, 128)
667 (80, 128)
668 (80, 128)
669 (80, 128)
670 (80, 128)
671 (80, 128)
672 (80, 128)
673 (80, 128)
674 (80, 128)
675 (80, 128)
676 (80, 128)
677 (80, 128)
678 (80, 128)
679 (80, 128)
680 (80, 128)
681 (80, 128)
682 (80, 128)
683 (80, 128)
684 (80, 128)
685 (80, 128)
686 (80, 128)
687 (80, 128)
688 (80, 128)
689 (80, 128)
690 (80, 128)
691 (80, 128)
692 (80, 128)
693 (80, 128)
694 (80, 128)
695 (80, 128)
696 (80, 128)
697 (80, 128)
698 (80, 128)
699 (80, 128)
700 (80, 128)
701 (80, 128)
702 (80, 128)
703 (80, 128)
704 (80, 128)
705 (80, 128)
706 (8

1259 (80, 128)
1260 (80, 128)
1261 (80, 128)
1262 (80, 128)
1263 (80, 128)
1264 (80, 128)
1265 (80, 128)
1266 (80, 128)
1267 (80, 128)
1268 (80, 128)
1269 (80, 128)
1270 (80, 128)
1271 (80, 128)
1272 (80, 128)
1273 (80, 128)
1274 (80, 128)
1275 (80, 128)
1276 (80, 128)
1277 (80, 128)
1278 (80, 128)
1279 (80, 128)
1280 (80, 128)
1281 (80, 128)
1282 (80, 128)
1283 (80, 128)
1284 (80, 128)
1285 (80, 128)
1286 (80, 128)
1287 (80, 128)
1288 (80, 128)
1289 (80, 128)
1290 (80, 128)
1291 (80, 128)
1292 (80, 128)
1293 (80, 128)
1294 (80, 128)
1295 (80, 128)
1296 (80, 128)
1297 (80, 128)
1298 (80, 128)
1299 (80, 128)
1300 (80, 128)
1301 (80, 128)
1302 (80, 128)
1303 (80, 128)
1304 (80, 128)
1305 (80, 128)
1306 (80, 128)
1307 (80, 128)
1308 (80, 128)
1309 (80, 128)
1310 (80, 128)
1311 (80, 128)
1312 (80, 128)
1313 (80, 128)
1314 (80, 128)
1315 (80, 128)
1316 (80, 128)
1317 (80, 128)
1318 (80, 128)
1319 (80, 128)
1320 (80, 128)
1321 (80, 128)
1322 (80, 128)
1323 (80, 128)
1324 (80, 128)
1325 (80, 

1820 (80, 128)
1821 (80, 128)
1822 (80, 128)
1823 (80, 128)
1824 (80, 128)
1825 (80, 128)
1826 (80, 128)
1827 (80, 128)
1828 (80, 128)
1829 (80, 128)
1830 (80, 128)
1831 (80, 128)
1832 (80, 128)
1833 (80, 128)
1834 (80, 128)
1835 (80, 128)
1836 (80, 128)
1837 (80, 128)
1838 (80, 128)
1839 (80, 128)


KeyboardInterrupt: 

In [126]:
# Replacing embedding of all zeros for UNK with a random embedding
_ = torch.nn.init.normal(text.vocab.vectors[0], mean=0, std=0.05) 
_ = torch.nn.init.normal(summary.vocab.vectors[0], mean=0, std=0.05)

In [127]:
# Input vocabulary
input_vocab = text.vocab
# Number of hidden units in each hidden layer
encoder_hidden_size = 512
# Number of recurrent layers in encoder
encoder_num_layers = 2
# Encoder dropout
encoder_dropout = 0

class EncoderRNN(nn.Module):
    
    def __init__(self, vocab, hidden_size, num_layers, dropout):
        super(EncoderRNN, self).__init__()
        
        # Hyperparameters
        self._vocab = vocab
        self._hidden_size = hidden_size
        self._num_layers = num_layers
        self._dropout = dropout
        
        # Hidden layer and cell state of model
        # Initialize before calling model
        self.hidden = None
        
        # Lookup table that stores word embeddings
        self.embed = nn.Embedding(len(self._vocab), num_dims).cuda()
        self.embed.weight.data.copy_(self._vocab.vectors)
        self.embed.weight.requires_grad = False
        
        # Pytorch lstm module
        self.lstm = nn.LSTM(num_dims, self._hidden_size, 
                            self._num_layers, dropout=self._dropout)
        self.lstm.cuda()
        
    def init_hidden(self, batch_size):
        return (Variable(torch.cuda.FloatTensor(self._num_layers, batch_size,
                    self._hidden_size).fill_(0), requires_grad=False), 
                Variable(torch.cuda.FloatTensor(self._num_layers, batch_size, 
                    self._hidden_size).fill_(0), 
                         requires_grad=False))
    
    def forward(self, batch_data, sequence_lengths):
        batch_size = len(sequence_lengths)
        # Embedding lookup
        input = self.embed(batch_data)
        # packed_input is of size Txbx*
        # where T is the length of longest sequence
        # b is batch size
        # batch is sorted in descending order of sequence lengths
        packed_input = pack_padded_sequence(input, list(sequence_lengths))
        packed_output, self.hidden = self.lstm(packed_input, self.hidden)
        # Final hidden state
        return self.hidden
    
encoder = EncoderRNN(input_vocab, encoder_hidden_size, 
                     encoder_num_layers, encoder_dropout)
encoder.cuda()

EncoderRNN(
  (embed): Embedding(114618, 300)
  (lstm): LSTM(300, 512, num_layers=2)
)

In [128]:
# Output vocabulary
output_vocab = summary.vocab
# Number of hidden units in each hidden layer
decoder_hidden_size = 512
# Number of recurrent layers in encoder
decoder_num_layers = 2
# Encoder dropout
decoder_dropout = 0

class DecoderRNN(nn.Module):
    
    def __init__(self, vocab, hidden_size, num_layers, dropout):
        super(DecoderRNN, self).__init__()
        
        # Hyperparameters
        self._vocab = vocab
        self._hidden_size = hidden_size
        self._num_layers = num_layers
        self._dropout = dropout
        
        # Lookup table that stores word embeddings
        self.embed = nn.Embedding(len(self._vocab), num_dims).cuda()
        self.embed.weight.data.copy_(self._vocab.vectors)
        self.embed.weight.requires_grad = False
        
        # Pytorch lstm cell for first layer
        self.lstmCell1 = nn.LSTMCell(num_dims, self._hidden_size)
        self.lstmCell1.cuda()
        
        # Pytorch lstm cell for second layer
        self.lstmCell2 = nn.LSTMCell(self._hidden_size, self._hidden_size)
        self.lstmCell2.cuda()
        
        # Linear transformation 
        self.linear_transform = nn.Linear(self._hidden_size, len(self._vocab))
    
    def forward(self, input, hidden1, hidden2):
        input = self.embed(input) ###
        hidden1 = self.lstmCell1(input, hidden1) 
        hidden2 = self.lstmCell2(hidden1[0], hidden2)
        output = self.linear_transform(hidden2[0])
        return hidden1, hidden2, output
    
decoder = DecoderRNN(output_vocab, decoder_hidden_size, 
                     decoder_num_layers, decoder_dropout)
decoder.cuda()

DecoderRNN(
  (embed): Embedding(43919, 300)
  (lstmCell1): LSTMCell(300, 512)
  (lstmCell2): LSTMCell(512, 512)
  (linear_transform): Linear(in_features=512, out_features=43919, bias=True)
)

In [129]:
def sample_distribution(distribution):
# Sample one element from a distribution assumed to be an array of normalized probabilities
    r = random.uniform(0, 1)
    s = 0
    for i in range(len(distribution)):
        s += distribution[i]
        if s >= r:
            return i
    return len(distribution) - 1

def predict_by_sampling(output, batch_size):
    next_input = torch.cuda.LongTensor(batch_size)
    # output has shape (batch_size, vocab_size)
    softmax = nn.Softmax(dim=1)
    output = softmax(output)
    for b in range(batch_size): ### Slow
        next_input[b] = sample_distribution(output[b].data)
    return Variable(next_input)

def most_likely(output, batch_size):
    if batch_size > 1:
        _, next_input = torch.topk(output, 1, dim=1)
    else: 
        _, next_input = torch.topk(output, 1)
    return next_input

In [130]:
learning_rate = 0.001

# Filter parameters that do not require gradients
encoder_parameters = filter(lambda p: p.requires_grad, encoder.parameters())
decoder_parameters = filter(lambda p: p.requires_grad, decoder.parameters())
# Optimizers
encoder_optimizer = torch.optim.SGD(encoder_parameters, lr=learning_rate)
decoder_optimizer = torch.optim.SGD(decoder_parameters, lr=learning_rate)
# Loss function
# By default the losses are averaged over observations for each minibatch
# When reduce is False returns a loss per batch element instead  
loss_function = nn.CrossEntropyLoss(reduce=False)

def train_model(batch):
    loss = 0
    # Clear model gradients
    encoder.zero_grad()
    decoder.zero_grad()
    # Clear optimizer gradients
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()
    # Clear hidden state of LSTM
    encoder.hidden = encoder.init_hidden(batch_size)
    # articles, titles are LongTensor vairables of shape (max_sequence_length, batch_size)
    # containig word indices from the respective vocabs
    # lengths are LongTensor varibles of shape batch_size containing
    # lengths of all the sequences in the batch
    articles, art_lengths = batch.article
    titles, tit_lengths = batch.title
    hiddenT = encoder(articles, art_lengths)
    # Seperate hidden states corresponding to the the two layers of the encoder
    hidden1 = (hiddenT[0][0], hiddenT[1][0])
    hidden2 = (hiddenT[0][1], hiddenT[1][1])
    # Initial input is a tensor of all <pad> tokens with zero embeddings
    input = Variable(torch.cuda.LongTensor(batch_size).fill_(1)) # 1 is the index of <pad>
    # Looping over all the sequences
    for t in range(torch.max(tit_lengths)):
        hidden1, hidden2, output = decoder(input, hidden1, hidden2)
        
        #input = predict_by_sampling(output, batch_size)
        input = most_likely(output, batch_size)
        
        # Cross entropy loss between words in title and predicted word
        loss_per_batch_element = loss_function(output, titles[t])
        # Sum over losses from the batch elements ignoring pad tokens
        for b in range(batch_size):
            if t < tit_lengths[b]: 
                loss += loss_per_batch_element[b]
    loss.backward()
    encoder_optimizer.step()
    decoder_optimizer.step()
    
    return loss/torch.sum(tit_lengths)

In [131]:
cudnn.benchmark = True
cudnn.fasttest = True

encoder.train()
decoder.train()
step = 1

for batch in train_iter:
    if torch.max(batch.article[1]) > 80: 
        step += 1
        continue
    loss = train_model(batch)
    if step % 100 == 0:
        print('Average minibatch loss at step %d: %.3f' % (step, loss))
    step += 1
            

Average minibatch loss at step 100: 8.766
Average minibatch loss at step 200: 9.968
Average minibatch loss at step 300: 7.916
Average minibatch loss at step 400: 7.624
Average minibatch loss at step 500: 7.572
Average minibatch loss at step 600: 7.789
Average minibatch loss at step 700: 7.576
Average minibatch loss at step 800: 7.443
Average minibatch loss at step 900: 7.467
Average minibatch loss at step 1100: 7.616
Average minibatch loss at step 1200: 7.504
Average minibatch loss at step 1300: 7.520
Average minibatch loss at step 1400: 7.535
Average minibatch loss at step 1500: 7.478
Average minibatch loss at step 1600: 7.365
Average minibatch loss at step 1700: 7.473
Average minibatch loss at step 1800: 7.404
Average minibatch loss at step 1900: 7.503
Average minibatch loss at step 2000: 7.413
Average minibatch loss at step 2100: 7.399
Average minibatch loss at step 2200: 7.302
Average minibatch loss at step 2300: 7.389
Average minibatch loss at step 2400: 7.283
Average minibatch lo

KeyboardInterrupt: 

for name, param in encoder.named_parameters():
    if param.requires_grad:
        print(name, param.data)